In [1]:
import os
import operator
import random
import numpy as np
import matplotlib.pyplot as plt
from joblib import Parallel, delayed

In [2]:
CONSTANT_PROBABILITY = 0.3
OPERATOR_PROBABILITY = 0.6
OPERATOR_FIRST_PROBABILITY = 0.9
CONSTANT_RANGE = (-10, 10)

cache = {}

In [3]:
class Node:
    def evaluate(self, x):
        raise NotImplementedError("Must implement evaluate method")

    def __str__(self):
        raise NotImplementedError("Must implement __str__ method")

    def clone(self):
        raise NotImplementedError("Must implement clone method")

    def from_string(self, string):
        raise NotImplementedError("Must implement from_string method")

class OperatorNode(Node):
    # Adding all NumPy mathematical functions to OPERATORS
    OPERATORS = {
        '+': operator.add,
        '-': operator.sub,
        '*': operator.mul,
        '/': lambda a, b: a / b if b != 0 else 1,  # Protected division
        'sin': lambda x: np.sin(np.clip(x, -1e10, 1e10)),  # Clamp input to avoid invalid values
        'cos': lambda x: np.cos(np.clip(x, -1e10, 1e10)),  # Clamp input to avoid invalid values
        'tan': lambda x: np.tan(np.clip(x, -1e10, 1e10)),  # Clamp input to avoid invalid values
        'log': lambda x: np.log(np.clip(x, 1e-10, None)),  # Avoid log(0) and negative values
        'exp': lambda x: np.exp(np.clip(x, -700, 700)),
        'sqrt': lambda x: np.sqrt(np.clip(x, 0, None)),  # Avoid sqrt of negative values
        'abs': lambda x: np.abs(x),
    }

    BINARY_OPERATORS = {'+', '-', '*', '/'}

    def __init__(self, operator_symbol, left=None, right=None, depth=0):
        self.operator_symbol = operator_symbol
        self.left = left
        self.right = right
        self.function = self.OPERATORS.get(operator_symbol)
        self.depth = depth

    def evaluate(self, x):
        # Evaluate left and right children
        left_val = self.left.evaluate(x)
        if self.operator_symbol not in self.BINARY_OPERATORS:
            result = self.function(left_val)
        else:
            right_val = self.right.evaluate(x)
            result = self.function(left_val, right_val)

        return result

    def __str__(self):
        if not self.right:
            return f"{self.operator_symbol}({self.left})"
        return f"({self.left} {self.operator_symbol} {self.right})"

    def clone(self):
        return OperatorNode(
            self.operator_symbol,
            self.left.clone() if self.left else None,
            self.right.clone() if self.right else None,
            self.depth,
        )

    def get_depth(self):
        left_depth = self.left.get_depth() if self.left else 0
        right_depth = self.right.get_depth() if self.right else 0
        return max(left_depth, right_depth)

    def from_string(self, string):
        # Parse the string to reconstruct the tree
        stack = []
        tokens = string.replace('(', ' ( ').replace(')', ' ) ').split()
        for token in tokens:
            if token == '(':
                continue
            elif token == ')':
                right = stack.pop()
                left = stack.pop()
                op = stack.pop()
                node = OperatorNode(op)
                node.left = left
                node.right = right
                stack.append(node)
            elif token in OperatorNode.OPERATORS:
                stack.append(token)
            else:
                stack.append(OperandNode(float(token)) if token.replace('.', '', 1).isdigit() else OperandNode(token))
        return stack[0]


class OperandNode(Node):
    def __init__(self, value, depth=0):
        self.value = value  # Can be a constant or a variable
        self.depth = depth

    def evaluate(self, x):
        # Evaluate the operand (constant or variable)
        if isinstance(self.value, str):
            result = x[int(self.value.lstrip('x_'))]
        else:
            result = self.value

        return result

    def __str__(self):
        return str(self.value)

    def clone(self):
        return OperandNode(self.value, self.depth)

    def get_depth(self):
        return self.depth + 1

    def from_string(self, string):
        return OperandNode(float(string) if string.replace('.', '', 1).isdigit() else string)

In [4]:
def generate_random_tree(max_depth, n_variables, current_depth=0, op=None):
    def get_operator():
        new_operator = random.choice(list(OperatorNode.OPERATORS.keys()))
        left = generate_random_tree(max_depth, n_variables, current_depth + 1, new_operator)
        right = None
        if new_operator in OperatorNode.BINARY_OPERATORS:
            right = generate_random_tree(max_depth, n_variables, current_depth + 1, new_operator)
        new_node = OperatorNode(new_operator, left, right, current_depth)
        return new_node


    def get_operand():
        if random.random() < CONSTANT_PROBABILITY:
            value = random.uniform(*CONSTANT_RANGE)
            value = round(value, 2)
            return OperandNode(value, current_depth)
        else:
            return OperandNode('x_' + str(random.randint(0, n_variables-1)), current_depth)


    if current_depth >= max_depth:
        return get_operand()

    elif current_depth == 0 and random.random() < OPERATOR_FIRST_PROBABILITY:
        return get_operator()

    elif random.random() < OPERATOR_PROBABILITY:
        return get_operator()

    return get_operand()


In [5]:
def get_all_nodes(node):
    nodes = [node]
    if isinstance(node, OperatorNode):
        nodes += get_all_nodes(node.left)
        nodes += get_all_nodes(node.right)

    return nodes

In [6]:
def initialize_population(pop_size, n_variables, max_depth=5):
    population = np.array([generate_random_tree(max_depth, n_variables) for _ in range(pop_size)])
    return population

In [7]:
def clear_cache():
    cache.clear()


def evaluate_individual(individual, x):
    key = (individual, tuple(x))
    if key in cache:
        return cache[key]
    result = individual.evaluate(x)
    cache[key] = result
    return result


def dominates(ind1: tuple[float, float], ind2: tuple[float, float]) -> bool:
    return (ind1[0] <= ind2[0] and ind1[1] <= ind2[1]) and (ind1[0] < ind2[0] or ind1[1] < ind2[1])


def pareto_front(population: list[Node], objectives: list[tuple[float, int]]) -> list[Node]:
    front = []
    for i, obj1 in enumerate(objectives):
        dominated = False
        for j, obj2 in enumerate(objectives):
            if i != j and dominates(obj2, obj1):
                dominated = True
                break
        if not dominated:
            front.append(population[i])
    return front


def diversity_preserving_selection(front: list[Node], objectives: list[tuple[float, int]], count: int) -> list[Node]:
    distances = [0] * len(front)

    # Calculate crowding distance for each objective
    for i, obj_values in enumerate(zip(*objectives)):
        sorted_indices = sorted(range(len(front)), key=lambda x: obj_values[x])
        distances[sorted_indices[0]] = distances[sorted_indices[-1]] = float('inf')
        for j in range(1, len(front) - 1):
            distances[sorted_indices[j]] += obj_values[sorted_indices[j + 1]] - obj_values[sorted_indices[j - 1]]

    # Select individuals with the highest crowding distance
    sorted_indices = sorted(range(len(front)), key=lambda x: distances[x], reverse=True)
    return [front[i] for i in sorted_indices[:count]]


def get_objectives(individual: Node, X: np.ndarray, y: np.ndarray) -> tuple[Node, float, int]:
    # Evaluate predictions for all samples in X
    predictions = np.array([evaluate_individual(individual, tuple(x)) for x in X])

    predictions_clipped = np.clip(predictions, -1e10, 1e10)

    # Calculate mean squared error
    mse = np.mean((y - predictions_clipped) ** 2)

    # Calculate complexity (number of nodes in the tree)
    complexity = len(get_all_nodes(individual))

    return individual, mse, complexity


def multi_objective_selection(objectives: dict) -> list[Node]:
    selected = []
    remaining_pop = list(objectives.keys())
    remaining_objectives = list(objectives.values())
    pop_size = len(remaining_pop)

    while len(selected) < pop_size:
        # Find the next Pareto front
        front = pareto_front(remaining_pop, remaining_objectives)

        if len(selected) + len(front) <= pop_size:
            selected.extend(front)
        else:
            # Use diversity-preserving selection if the front size exceeds remaining slots
            selected.extend(diversity_preserving_selection(front, remaining_objectives, pop_size - len(selected)))
            break

        # Remove Pareto front solutions from the remaining population
        for individual in front:
            idx = remaining_pop.index(individual)
            del remaining_pop[idx]
            del remaining_objectives[idx]

    return selected



In [8]:
def crossover(parent1: Node, parent2: Node) -> tuple[Node, Node]:
    parent1 = parent1.clone()
    parent2 = parent2.clone()

    # Get all nodes from both parents
    nodes1 = get_all_nodes(parent1)
    nodes2 = get_all_nodes(parent2)

    # Randomly select crossover points
    crossover_point1 = random.choice(nodes1)
    while crossover_point1 is None:
        crossover_point1 = random.choice(nodes1)

    crossover_point2 = random.choice(nodes2)
    while crossover_point2 is None:
        crossover_point2 = random.choice(nodes2)

    # Swap the subtrees if both crossover points are OperatorNodes
    if isinstance(crossover_point1, OperatorNode) and isinstance(crossover_point2, OperatorNode):
        swap_subtrees(crossover_point1, crossover_point2)
    elif isinstance(crossover_point1, OperandNode) and isinstance(crossover_point2, OperandNode):
        swap_operands(crossover_point1, crossover_point2)
    else:
        # Replace the subtree of one parent with the subtree of the other parent
        replace_subtree(parent1, parent2, crossover_point1, crossover_point2)

    return parent1, parent2


def swap_subtrees(node1: OperatorNode, node2: OperatorNode):
    node1.left, node2.left = node2.left.clone(), node1.left.clone()
    if node1.right is not None:
        if node2.right is not None:
            node1.right, node2.right = node2.right.clone(), node1.right.clone()
        else:
            node2.right = node1.right.clone()
            node1.right = None
    else:
        if node2.right is not None:
            node1.right = node2.right.clone()
            node2.right = None

    node1.operator_symbol, node2.operator_symbol = node2.operator_symbol, node1.operator_symbol
    node1.function, node2.function = node2.function, node1.function


def swap_operands(node1: OperandNode, node2: OperandNode):
    node1.value, node2.value = node2.value, node1.value


def replace_subtree(ind1, ind2, subtree1: Node, subtree2: Node):
    if subtree1 is None or subtree2 is None:
        raise ValueError("Cannot replace subtree: node is None.")

    parent1 = find_parent(ind1, subtree1)
    parent2 = find_parent(ind2, subtree2)

    if parent1 is None or parent2 is None:
        return

    if parent1.left is subtree1 and parent2.left is subtree2:
        parent1.left = subtree2.clone()
        parent2.left = subtree1.clone()
    elif parent1.right is subtree1 and parent2.right is subtree2:
        parent1.right = subtree2.clone()
        parent2.right = subtree1.clone()
    elif parent1.left is subtree1 and parent2.right is subtree2:
        parent1.left = subtree2.clone()
        parent2.right = subtree1.clone()
    elif parent1.right is subtree1 and parent2.left is subtree2:
        parent1.right = subtree2.clone()
        parent2.left = subtree1.clone()

In [9]:
def find_parent(root: Node, target: Node, parent: Node = None) -> Node | None:
    if root is target:
        return parent

    if isinstance(root, OperatorNode):
        if root.left:
            found = find_parent(root.left, target, root)
            if found:
                return found
        if root.right:
            found = find_parent(root.right, target, root)
            if found:
                return found

    return None


def replace_child(root: Node, old_child: Node, new_child: Node) -> None:
    if not isinstance(root, OperatorNode):
        return

    if root.left is old_child:
        root.left = new_child
        return
    if root.right is old_child:
        root.right = new_child
        return

    # Recursively check the children
    if root.left:
        replace_child(root.left, old_child, new_child)
    if root.right:
        replace_child(root.right, old_child, new_child)


def mutate(individual: Node, max_depth: int, mutation_rate: float, n_variables: int) -> Node:
    if random.random() >= mutation_rate:
        return individual  # No mutation occurs

    # Clone the individual to avoid modifying the original
    individual = individual.clone()

    # Get all nodes in the tree
    nodes = get_all_nodes(individual)
    if not nodes:
        raise ValueError("Cannot mutate: The tree has no nodes.")

    # Select a random node for mutation
    mutate_node = random.choice(nodes)
    while mutate_node is None:
        mutate_node = random.choice(nodes)

    # Apply mutation strategies
    mutation_type = random.choice(["shrink", "subtree_replacement", "hoist", "tweak"])
    if mutation_type == "shrink":
        return apply_shrink_mutation(individual, mutate_node, n_variables)
    elif mutation_type == "subtree_replacement" and max_depth - mutate_node.depth > 1:
        new_subtree = generate_random_tree(max_depth - mutate_node.depth, n_variables, mutate_node.depth)
        replace(individual, mutate_node, new_subtree)
    elif mutation_type == "hoist":
        return apply_hoist_mutation(individual, mutate_node)
    elif mutation_type == "tweak":
        tweak(mutate_node, n_variables)

    return individual


def apply_shrink_mutation(individual: Node, mutate_node: Node, n_variables: int) -> Node:
    # Find the parent of the mutate_node
    parent = find_parent(individual, mutate_node)

    # Generate a terminal node (variable or constant)
    terminal = OperandNode(
        value=random.choice([random.uniform(-10, 10), f"x_{random.randint(0, n_variables - 1)}"]),
        depth=mutate_node.depth  # Preserve depth for consistency
    )

    # Replace the mutate_node with the terminal node
    if parent:
        replace_child(parent, mutate_node, terminal)
    else:
        # If no parent, mutate_node is the root, replace the root directly
        individual = terminal

    return individual


def replace(individual, node: Node, new_subtree: Node):
    if node is None or new_subtree is None:
        raise ValueError("Cannot replace subtree: node is None.")

    parent = find_parent(individual, node)

    if parent is None:
        return

    if parent.left is node:
        parent.left = new_subtree.clone()
    elif parent.right is node:
        parent.right = new_subtree.clone()


def apply_hoist_mutation(individual: Node, mutate_node: Node) -> Node:
    # Ensure the mutate_node has at least one child to hoist
    if not isinstance(mutate_node, OperatorNode) or not mutate_node.left:
        return individual  # No hoist possible, return unchanged

    # Select the subtree to promote (left child by default)
    hoisted_subtree = mutate_node.left

    # Find the parent of the mutate_node
    parent = find_parent(individual, mutate_node)

    if parent:
        # Replace the mutate_node with the hoisted subtree in the parent's reference
        replace_child(parent, mutate_node, hoisted_subtree)
    else:
        # If the mutate_node is the root, replace the entire tree
        individual = hoisted_subtree

    return individual


def tweak(node: Node, n_variables: int):
    if isinstance(node, OperatorNode):
        mutate_operator(node)
    elif isinstance(node, OperandNode):
        mutate_operand(node, n_variables)


def mutate_operator(node: OperatorNode):
    available_operators = []
    if node.operator_symbol not in OperatorNode.BINARY_OPERATORS:
        # Unary operator
        available_operators = [op for op in OperatorNode.OPERATORS.keys() if op not in OperatorNode.BINARY_OPERATORS]
        available_operators.remove(node.operator_symbol)
    else:
        # Binary operator
        available_operators = [op for op in OperatorNode.BINARY_OPERATORS]
        available_operators.remove(node.operator_symbol)

    node.operator_symbol = random.choice(available_operators)
    node.function = OperatorNode.OPERATORS[node.operator_symbol]


def mutate_operand(node: OperandNode, n_variables: int):
    if isinstance(node.value, float):
        # Mutate constant value slightly
        node.value += random.uniform(-1, 1)
    elif isinstance(node.value, str) and node.value.startswith('x_'):
        # Change the variable index
        node.value = f"x_{random.randint(0, n_variables - 1)}"

In [10]:
def simplify_expression(node: Node) -> Node:
    if isinstance(node, OperandNode):
        return node

    if isinstance(node, OperatorNode):
        # Recursively simplify left and right subtrees
        node.left = simplify_expression(node.left)
        if node.right:
            node.right = simplify_expression(node.right)

        # Simplify based on operator type
        if node.operator_symbol in OperatorNode.BINARY_OPERATORS:
            node = simplify_binary_operator(node)
        else:
            node = simplify_unary_operator(node)

    return node


def simplify_binary_operator(node: OperatorNode) -> Node:
    if isinstance(node.left, OperandNode) and isinstance(node.right, OperandNode):
        if isinstance(node.left.value, float) and isinstance(node.right.value, float):
            # Both children are constants; compute the result
            return OperandNode(node.function(node.left.value, node.right.value))

    # Apply simplification rules for binary operators
    if node.operator_symbol == '+':
        if isinstance(node.left, OperandNode) and node.left.value == 0:
            return node.right
        if isinstance(node.right, OperandNode) and node.right.value == 0:
            return node.left
    elif node.operator_symbol == '*':
        if isinstance(node.left, OperandNode) and node.left.value == 1:
            return node.right
        if isinstance(node.right, OperandNode) and node.right.value == 1:
            return node.left
        if isinstance(node.left, OperandNode) and node.left.value == 0:
            return OperandNode(0)
        if isinstance(node.right, OperandNode) and node.right.value == 0:
            return OperandNode(0)
    elif node.operator_symbol == '-':
        if isinstance(node.right, OperandNode) and node.right.value == 0:
            return node.left
        if isinstance(node.left, OperandNode) and isinstance(node.right, OperandNode) and node.left.value == node.right.value:
            return OperandNode(0)
    elif node.operator_symbol == '/':
        if isinstance(node.right, OperandNode) and node.right.value == 1:
            return node.left
        if isinstance(node.left, OperandNode) and isinstance(node.right, OperandNode) and node.left.value == node.right.value:
            return OperandNode(1)
    elif node.operator_symbol == '**':
        if isinstance(node.right, OperandNode) and node.right.value == 1:
            return node.left
        if isinstance(node.right, OperandNode) and node.right.value == 0:
            return OperandNode(1)

    return node


def simplify_unary_operator(node: OperatorNode) -> Node:
    if isinstance(node.left, OperandNode):
        if isinstance(node.left.value, float):
            # Compute the result for unary operators
            return OperandNode(node.function(node.left.value))

    return node


In [12]:
def trim_population(population: list[Node]) -> list[Node]:
    unique_individuals = []
    seen_hashes = set()

    for ind in population:
        # Use a hash of the string representation for uniqueness
        ind_hash = hash(str(ind))
        if ind_hash not in seen_hashes:
            seen_hashes.add(ind_hash)
            unique_individuals.append(ind)

    return unique_individuals

In [13]:
import time

def genetic_programming(
        X, y,
        pop_size,
        generations,
        max_depth,
        crossover_rate,
        mutation_rate,
        elitism=True,
        elitism_size=3,
        max_no_improvement=5,
        verbose=True,
):
    start_time = time.time()

    n_variables = len(X[0])

    population = initialize_population(pop_size, n_variables, max_depth)


    if verbose:
        init_pop_time = time.time()
        print(f"Population Init Time: {init_pop_time - start_time:.6f}")

    best_fitness_values = []
    best_individuals = []

    gens_without_improvement = 0

    for gen in range(generations):
        clear_cache()
        init_pop_time = time.time()

        objectives = Parallel(n_jobs=-1)(delayed(get_objectives)(ind, X, y) for ind in population)
        objectives_dict = {ind: (mse, complexity) for ind, mse, complexity in sorted(objectives, key=lambda x: (x[1], x[2]))}

        if verbose:
            fitnesses_evaluation_time = time.time()
            print(f"Fitness Evaluation Time: {fitnesses_evaluation_time - init_pop_time:.6f}")

        # Track the best individual
        best_individual = list(objectives_dict.keys())[0]
        best_fitness = objectives_dict[best_individual][0]

        if len(best_fitness_values) > 0 and best_fitness_values[-1] == best_fitness:
            gens_without_improvement += 1
        else:
            gens_without_improvement = 0

        best_fitness_values.append(best_fitness)
        best_individuals.append(best_individual)

        print(f"Generation {gen+1}: Best Fitness = {best_fitness:.6f}")
        if best_fitness < 0.0001:
            return best_individual, best_fitness, best_fitness_values

        # Select individuals for the next generation
        selected = multi_objective_selection(objectives_dict)
        if verbose:
            selection_time = time.time()
            print(f"Selection Time: {selection_time - fitnesses_evaluation_time:.6f}")

        # Create next generation
        next_generation = []

        # Elitism
        if elitism:
            next_generation.extend(objectives_dict.keys()[:elitism_size])

        if gens_without_improvement > max_no_improvement:
            if elitism:
                next_generation.extend(objectives_dict.keys()[elitism_size:pop_size * 0.6])
            else:
                next_generation.extend(objectives_dict.keys()[:pop_size * 0.6])

            next_generation.extend(initialize_population(pop_size - len(next_generation), n_variables, max_depth))
            gens_without_improvement = 0
            continue

        if verbose:
            elitism_time = time.time()
            print(f"Elitism Time: {elitism_time - selection_time:.6f}")

        # Generate offspring
        while len(next_generation) < pop_size:
            if random.random() < crossover_rate:
                parent1, parent2 = random.sample(selected, 2)
                offspring1, offspring2 = crossover(parent1, parent2)
                next_generation.append(simplify_expression(offspring1))
                next_generation.append(simplify_expression(offspring2))
            else:
                individual = random.choice(selected)
                mutated = mutate(individual, max_depth, mutation_rate, n_variables)
                next_generation.append(simplify_expression(mutated))

        if verbose:
            new_generation_time = time.time()
            print(f"New Generation Time: {new_generation_time - elitism_time:.6f}")

        next_generation = trim_population(next_generation)
        if verbose:
            trimming_time = time.time()
            print(f"Trimming Time: {trimming_time - new_generation_time:.6f}")

        if random.random() < 0.2:
            if verbose:
                print("Trimming Population")

            next_generation = next_generation[:int(0.6 * pop_size)]

        while len(next_generation) < pop_size:
            next_generation.append(generate_random_tree(max_depth, n_variables))

        if verbose:
            random_generation_time = time.time()
            print(f"Random Generation Time: {random_generation_time - trimming_time:.6f}")

        population = next_generation


    objectives = Parallel(n_jobs=-1)(delayed(get_objectives)(ind, X, y) for ind in population)
    objectives_dict = {}
    for ind, mse, complexity in objectives:
        objectives_dict[ind] = (mse, complexity)

    objectives_dict = {ind: objectives_dict[ind] for ind in sorted(objectives_dict.keys(), key=lambda ind: objectives_dict[ind])}

    best_individual = list(objectives_dict.keys())[0]
    best_fitness = objectives_dict[best_individual][0]

    best_fitness_values.append(best_fitness)
    best_individuals.append(best_individual)

    # Final evaluation
    best_fitness = min(best_fitness_values)
    best_index = best_fitness_values.index(best_fitness)
    best_individual = best_individuals[best_index]

    if verbose:
        print(f"\nBest Overall Fitness: {best_fitness:.6f}")

    return best_individual, best_fitness, best_fitness_values

In [16]:
def run():
    np.random.seed(41)
    random.seed(41)
    problem = np.load('data/problem_8.npz')
    X = np.array(problem['x'])
    y = np.array(problem['y'])

    # Run Genetic Programming
    best_expr, best_fit, best_fitness_values = genetic_programming(
        X.T, y,
        pop_size=500,
        generations=500,
        max_depth=5,
        crossover_rate=0.4,
        mutation_rate=0.8,
        elitism=True,
        elitism_size=10,
        verbose=False,
    )

    print(f"\nBest Expression: {best_expr}")

    # Generate predictions
    y_pred = np.array([best_expr.evaluate(x) for x in X.T])

    # Visualize the results considering X has a variable number of arrays inside it
    if X.shape[0] == 1:
        plt.figure(figsize=(10, 6))
        plt.plot(X[0], y, color='blue', label='Data', alpha=0.6)
        plt.plot(X[0], y_pred, color='red', label='Best GP Expression')
        plt.legend()
        plt.title('Symbolic Regression using Genetic Programming')
        plt.xlabel('x')
        plt.ylabel('y')
        plt.show()
    else:
        fig = plt.figure(figsize=(10, 6))
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter(X[0], X[1], y, color='blue', label='Data', alpha=0.6)
        ax.plot_trisurf(X[0], X[1], y_pred, color='red', alpha=0.6)
        ax.set_xlabel('x_0')
        ax.set_ylabel('x_1')
        ax.set_zlabel('y')
        ax.set_title('Symbolic Regression using Genetic Programming (3D Visualization)')
        plt.legend()
        plt.title(f'Symbolic Regression using Genetic Programming')
        plt.show()

    plt.figure(figsize=(10, 6))
    plt.plot(best_fitness_values, color='blue', label='Best Fitness')
    plt.title('Best Fitness over Generations')
    plt.xlabel('Generation')
    plt.ylabel('Best Fitness')
    plt.legend()
    plt.show()

In [17]:
run()

Generation 1: Best Fitness = 22667096.904650
Generation 2: Best Fitness = 15327540.516692
Generation 3: Best Fitness = 15327540.516692
Generation 4: Best Fitness = 15327540.516692
Generation 5: Best Fitness = 15327540.516692
Generation 6: Best Fitness = 15327540.516692
Generation 7: Best Fitness = 15327540.516692
Generation 8: Best Fitness = 15327540.516692
Generation 9: Best Fitness = 15327540.516692
Generation 10: Best Fitness = 15327540.516692
Generation 11: Best Fitness = 15327540.516692


/var/folders/db/psm35sh503x7y51vnw94qbzw0000gn/T/ipykernel_19030/4114134282.py:20: RuntimeWarning: overflow encountered in scalar divide


Generation 12: Best Fitness = 15327540.516692
Generation 13: Best Fitness = 15327540.516692
Generation 14: Best Fitness = 15327540.516692
Generation 15: Best Fitness = 15327540.516692
Generation 16: Best Fitness = 15327540.516692
Generation 17: Best Fitness = 15327540.516692
Generation 18: Best Fitness = 15327540.516692
Generation 19: Best Fitness = 14004834.794078
Generation 20: Best Fitness = 14004834.794078
Generation 21: Best Fitness = 14004834.794078
Generation 22: Best Fitness = 14004834.794078
Generation 23: Best Fitness = 14004834.794078
Generation 24: Best Fitness = 13682683.260043
Generation 25: Best Fitness = 13682683.260043
Generation 26: Best Fitness = 13682683.260043


/var/folders/db/psm35sh503x7y51vnw94qbzw0000gn/T/ipykernel_19030/4114134282.py:20: RuntimeWarning: overflow encountered in scalar divide


Generation 27: Best Fitness = 13682683.260043
Generation 28: Best Fitness = 13682683.260043
Generation 29: Best Fitness = 13682683.260043
Generation 30: Best Fitness = 13682683.260043
Generation 31: Best Fitness = 13682683.260043
Generation 32: Best Fitness = 13682683.260043
Generation 33: Best Fitness = 13682683.260043
Generation 34: Best Fitness = 13682683.260043
Generation 35: Best Fitness = 13682683.260043
Generation 36: Best Fitness = 13682683.260043
Generation 37: Best Fitness = 13682683.260043
Generation 38: Best Fitness = 13682683.260043
Generation 39: Best Fitness = 13682683.260043
Generation 40: Best Fitness = 13682683.260043
Generation 41: Best Fitness = 13682683.260043
Generation 42: Best Fitness = 13682683.260043
Generation 43: Best Fitness = 13682683.260043
Generation 44: Best Fitness = 13682683.260043
Generation 45: Best Fitness = 13682683.260043
Generation 46: Best Fitness = 13682683.260043
Generation 47: Best Fitness = 13682683.260043
Generation 48: Best Fitness = 1368

...

/var/folders/db/psm35sh503x7y51vnw94qbzw0000gn/T/ipykernel_19030/4114134282.py:20: RuntimeWarning: overflow encountered in scalar divide


Generation 86: Best Fitness = 11321745.807584
Generation 87: Best Fitness = 11321745.807584
Generation 88: Best Fitness = 11321745.807584
Generation 89: Best Fitness = 11134643.488144
Generation 90: Best Fitness = 11134643.488144
Generation 91: Best Fitness = 11134643.488144
Generation 92: Best Fitness = 11134643.488144


/var/folders/db/psm35sh503x7y51vnw94qbzw0000gn/T/ipykernel_19030/4114134282.py:20: RuntimeWarning: overflow encountered in scalar divide


Generation 93: Best Fitness = 11134643.488144
Generation 94: Best Fitness = 11134643.488144
Generation 95: Best Fitness = 11134643.488144
Generation 96: Best Fitness = 11134643.488144
Generation 97: Best Fitness = 11134643.488144
Generation 98: Best Fitness = 11134643.488144
Generation 99: Best Fitness = 11134643.488144
Generation 100: Best Fitness = 11134643.488144
Generation 101: Best Fitness = 11134643.488144
Generation 102: Best Fitness = 11134643.488144
Generation 103: Best Fitness = 11134643.488144
Generation 104: Best Fitness = 11134643.488144
Generation 105: Best Fitness = 11134643.488144
Generation 106: Best Fitness = 11134643.488144
Generation 107: Best Fitness = 11134643.488144
Generation 108: Best Fitness = 11134643.488144
Generation 109: Best Fitness = 11134643.488144
Generation 110: Best Fitness = 11134643.488144
Generation 111: Best Fitness = 11134643.488144


KeyboardInterrupt: 